## Explore BABEL

We present some code to explore BABEL by computing stats., and searching for specific actions.

In [1]:
# Preparing the environment
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os, pdb
from os.path import join as ospj
import json
from collections import *

import numpy as np
import pandas as pd
from pandas.core.common import flatten

import pprint
pp = pprint.PrettyPrinter()

### Load BABEL 
Note that we are not loading the test set 

In [2]:
d_folder = '../data/babel_v1.0_release' # Data folder
l_babel_dense_files = ['train', 'val']  
l_babel_extra_files = ['extra_train', 'extra_val']

# BABEL Dataset 
babel = {}
for file in l_babel_dense_files:
    babel[file] = json.load(open(ospj(d_folder, file+'.json')))
    
for file in l_babel_extra_files:
    babel[file] = json.load(open(ospj(d_folder, file+'.json'))) 

### Duration of mocap for which BABEL action labels are available

In [3]:
for babel_set in [l_babel_dense_files, l_babel_dense_files+l_babel_extra_files]:
    dur = 0.0
    list_sids = []    
    for spl in babel_set:
        for sid in babel[spl]:
            if sid not in list_sids:
                list_sids.append(sid)
                dur += babel[spl][sid]['dur'] 
                
    # Duration of each set
    minutes = dur//60
    print('Total duration = {0} hours {1} min. {2:.0f} sec.'.format(
                                            minutes//60, minutes%60, dur%60))
    print('Total # seqs. = ', len(list_sids))
    print('-'*30)

Total duration = 30.0 hours 2.0 min. 32 sec.
Total # seqs. =  8808
------------------------------
Total duration = 34.0 hours 43.0 min. 39 sec.
Total # seqs. =  10576
------------------------------


### Search BABEL for action

In [4]:
def get_cats(ann, file):
    # Get sequence labels and frame labels if they exist
    seq_l, frame_l = [], []
    if 'extra' not in file:
        if ann['seq_ann'] is not None:
            seq_l = flatten([seg['act_cat'] for seg in ann['seq_ann']['labels']])
        if ann['frame_ann'] is not None:
            frame_l = flatten([seg['act_cat'] for seg in ann['frame_ann']['labels']])
    else:
        # Load all labels from (possibly) multiple annotators
        if ann['seq_anns'] is not None:
            seq_l = flatten([seg['act_cat'] for seq_ann in ann['seq_anns'] for seg in seq_ann['labels']])
        if ann['frame_anns'] is not None:            
            frame_l = flatten([seg['act_cat'] for frame_ann in ann['frame_anns'] for seg in frame_ann['labels']])
            
    return list(seq_l), list(frame_l)

In [5]:
action = 'jump'
act_anns = defaultdict(list) # { seq_id_1: [ann_1_1, ann_1_2], seq_id_2: [ann_2_1], ...} 
n_act_spans = 0

for spl in babel:
    for sid in babel[spl]:
        
        seq_l, frame_l = get_cats(babel[spl][sid], spl)
        # print(seq_l + frame_l)
        
        if action in seq_l + frame_l:
            
            # Store all relevant mocap sequence annotations
            act_anns[sid].append(babel[spl][sid])
            
            # # Individual spans of the action in the sequence
            n_act_spans += Counter(seq_l+frame_l)[action]
            
print('# Seqs. containing action {0} = {1}'.format(action, len(act_anns)))
print('# Segments containing action {0} = {1}'.format(action, n_act_spans))

# Seqs. containing action jump = 746
# Segments containing action jump = 1597


In [6]:
# View a random annotation 
key = np.random.choice(list(act_anns.keys()))
pp.pprint(act_anns[key])

[{'babel_sid': 7692,
  'dur': 3.83,
  'feat_p': 'CMU/CMU/141/141_05_poses.npz',
  'frame_ann': {'anntr_id': 'eab5b72f-7399-43a7-a752-e4ee2807faaf',
                'babel_lid': '59ad905d-f378-4d2b-90a7-4e3222bbc1f7',
                'labels': [{'act_cat': ['hop'],
                            'end_t': 2,
                            'proc_label': 'hop left',
                            'raw_label': 'hopping left',
                            'seg_id': 'daf942ad-7cbe-4387-b6a0-0fc391c702ea',
                            'start_t': 1},
                           {'act_cat': ['hop'],
                            'end_t': 3,
                            'proc_label': 'hop right',
                            'raw_label': 'hopping right',
                            'seg_id': '7b17f75e-3da9-4e56-aca1-9bbb6b8d5dd9',
                            'start_t': 2},
                           {'act_cat': ['stand'],
                            'end_t': 1,
                            'proc_label': 'stand',
